# Import Libraries

In [1]:
import pandas as pd

from utils import clean_text, remove_numbers, remove_punctuations, remove_strings, remove_stopwords
from evaluation import multi_column_embedding_model_evaluation
from constants import (
    TEST_DATA_PATH,
    CLEANED_TEST_DATA_PATH,
    QWEN3_EMBEDDING_CONFIG_PATH,
    E5_LARGE_INSTRUCT_CONFIG_PATH,
    TRAIN_VAL_DATA_PATH,
    CLASSES_TRANSLATION,
    CLEANED_TRAIN_DATA_PATH,
)

c:\Users\na255073\Documents\Iter4-5\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\na255073\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Prepare Data

## Load Data

In [2]:
df_train = pd.read_csv(".." / TRAIN_VAL_DATA_PATH)
df_test = pd.read_csv(".." / TEST_DATA_PATH)
df_train.head(5)

,Item_Name,class,Brand,Weight,Number of units,Size of units,Price,T.Price,Pack,Unit
0,مونتانا ذره 400 ج,Vegetables & Fruits,مونتانا,400جم,1,NaN,NaN,NaN,كيس,جم
1,Ahmad Tea Fruit And Herb Selection Herbal Teab...,"Tea, Coffee & Hot Drinks",Ahmad Tea,NaN,20,NaN,NaN,NaN,علبة,NaN
2,Lulu Brown Samoon 1pkt,Bakery,Lulu,NaN,1,NaN,NaN,NaN,عبوة,NaN
3,فلفل رومى بلدى حشو وزن,Vegetables & Fruits,NaN,NaN,1,NaN,NaN,NaN,كيس,NaN
4,كانز,Soft Drinks & Juices,NaN,NaN,1,NaN,NaN,NaN,كانز,NaN


In [3]:
x = set(df_train["class"].dropna().tolist())

x = set([remove_punctuations(i.lower()) for i in x])
y = set(CLASSES_TRANSLATION.keys())

x.difference(y)

set()

In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42949 entries, 0 to 42948
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Item_Name        42944 non-null  object
 1   class            42938 non-null  object
 2   Brand            34485 non-null  object
 3   Weight           27267 non-null  object
 4   Number of units  42929 non-null  object
 5   Size of units    419 non-null    object
 6   Price            2398 non-null   object
 7   T.Price          2391 non-null   object
 8   Pack             41563 non-null  object
 9   Unit             27641 non-null  object
dtypes: object(10)
memory usage: 3.3+ MB


## Remove Nulls

In [5]:
df_train.dropna(subset=["Item_Name", "class"], inplace=True)
df_test.dropna(subset=["Item_Name", "class"], inplace=True)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42935 entries, 0 to 42948
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Item_Name        42935 non-null  object
 1   class            42935 non-null  object
 2   Brand            34483 non-null  object
 3   Weight           27260 non-null  object
 4   Number of units  42917 non-null  object
 5   Size of units    419 non-null    object
 6   Price            2397 non-null   object
 7   T.Price          2390 non-null   object
 8   Pack             41550 non-null  object
 9   Unit             27634 non-null  object
dtypes: object(10)
memory usage: 3.6+ MB


## Normalize Data

In [6]:
df_train["Item_Name"] = df_train["Item_Name"].str.lower()
df_train["class"] = df_train["class"].str.lower()
df_train["Unit"] = df_train["Unit"].str.lower()
df_train["Weight"] = df_train["Weight"].str.lower()
df_train["Brand"] = df_train["Brand"].str.lower()
df_test["Item_Name"] = df_test["Item_Name"].str.lower()
df_test["class"] = df_test["class"].str.lower()
df_test["Unit"] = df_test["Unit"].str.lower()
df_test["Weight"] = df_test["Weight"].str.lower()
df_test["Brand"] = df_test["Brand"].str.lower()
df_train.head()

,Item_Name,class,Brand,Weight,Number of units,Size of units,Price,T.Price,Pack,Unit
0,مونتانا ذره 400 ج,vegetables & fruits,مونتانا,400جم,1,NaN,NaN,NaN,كيس,جم
1,ahmad tea fruit and herb selection herbal teab...,"tea, coffee & hot drinks",ahmad tea,NaN,20,NaN,NaN,NaN,علبة,NaN
2,lulu brown samoon 1pkt,bakery,lulu,NaN,1,NaN,NaN,NaN,عبوة,NaN
3,فلفل رومى بلدى حشو وزن,vegetables & fruits,NaN,NaN,1,NaN,NaN,NaN,كيس,NaN
4,كانز,soft drinks & juices,NaN,NaN,1,NaN,NaN,NaN,كانز,NaN


## Clean Data

In [7]:
df_train["removed_punctuations"] = df_train["Item_Name"].apply(remove_punctuations)
df_train["removed_numbers"] = df_train["Item_Name"].apply(lambda x: remove_numbers(x, remove_string=False))
df_train["removed_brand"] = df_train.apply(lambda x: remove_strings(x.Item_Name, [x.Brand]), axis=1)
df_train["removed_pack"] = df_train.apply(lambda x: remove_strings(x.Item_Name, [x.Pack]), axis=1)
df_train["removed_unit"] = df_train.apply(lambda x: remove_strings(x.Item_Name, [x.Unit]), axis=1)
df_train["removed_numbers_and_punctuations"] = df_train["Item_Name"].apply(lambda x: remove_numbers(remove_punctuations(x), remove_string=False))
df_train["removed_stopwords"] = df_train["Item_Name"].apply(remove_stopwords)
df_train["class"] = df_train["class"].apply(remove_punctuations)
df_train["cleaned_text"] = df_train.apply(clean_text, axis=1)

df_test["removed_punctuations"] = df_test["Item_Name"].apply(remove_punctuations)
df_test["removed_numbers"] = df_test["Item_Name"].apply(lambda x: remove_numbers(x, remove_string=False))
df_test["removed_brand"] = df_test.apply(lambda x: remove_strings(x.Item_Name, [x.Brand]), axis=1)
df_test["removed_pack"] = df_test.apply(lambda x: remove_strings(x.Item_Name, [x.Pack]), axis=1)
df_test["removed_unit"] = df_test.apply(lambda x: remove_strings(x.Item_Name, [x.Unit]), axis=1)
df_test["removed_numbers_and_punctuations"] = df_test["Item_Name"].apply(lambda x: remove_numbers(remove_punctuations(x), remove_string=False))
df_test["removed_stopwords"] = df_test["Item_Name"].apply(remove_stopwords)
df_test["class"] = df_test["class"].apply(remove_punctuations)
df_test["cleaned_text"] = df_test.apply(clean_text, axis=1)
df_train.head()

,Item_Name,class,Brand,Weight,Number of units,Size of units,Price,T.Price,Pack,Unit,removed_punctuations,removed_numbers,removed_brand,removed_pack,removed_unit,removed_numbers_and_punctuations,removed_stopwords,cleaned_text
0,مونتانا ذره 400 ج,vegetables fruits,مونتانا,400جم,1,NaN,NaN,NaN,كيس,جم,مونتانا ذره 400 ج,مونتانا ذره ج,ذره 400 ج,مونتانا ذره 400 ج,مونتانا ذره 400 ج,مونتانا ذره ج,مونتانا ذره 400,ذره
1,ahmad tea fruit and herb selection herbal teab...,tea coffee hot drinks,ahmad tea,NaN,20,NaN,NaN,NaN,علبة,NaN,ahmad tea fruit and herb selection herbal teab...,ahmad tea fruit and herb selection herbal teab...,fruit and herb selection herbal teabags - 20 ...,ahmad tea fruit and herb selection herbal teab...,ahmad tea fruit and herb selection herbal teab...,ahmad tea fruit and herb selection herbal teab...,ahmad tea fruit herb selection herbal teabags ...,fruit herb selection herbal teabags pieces
2,lulu brown samoon 1pkt,bakery,lulu,NaN,1,NaN,NaN,NaN,عبوة,NaN,lulu brown samoon 1pkt,lulu brown samoon pkt,brown samoon 1pkt,lulu brown samoon 1pkt,lulu brown samoon 1pkt,lulu brown samoon pkt,lulu brown samoon 1pkt,brown samoon pkt
3,فلفل رومى بلدى حشو وزن,vegetables fruits,NaN,NaN,1,NaN,NaN,NaN,كيس,NaN,فلفل رومى بلدى حشو وزن,فلفل رومى بلدى حشو وزن,فلفل رومى بلدى حشو وزن,فلفل رومى بلدى حشو وزن,فلفل رومى بلدى حشو وزن,فلفل رومى بلدى حشو وزن,فلفل رومى بلدى حشو وزن,فلفل رومى بلدى حشو وزن
4,كانز,soft drinks juices,NaN,NaN,1,NaN,NaN,NaN,كانز,NaN,كانز,كانز,كانز,,كانز,كانز,كانز,كانز


In [8]:
df_train[df_train["cleaned_text"]==""]
df_test[df_test["cleaned_text"]==""]

,Item_Name,class,Brand,Weight,Number of units,Size of units,Price,T.Price,Pack,Unit,removed_punctuations,removed_numbers,removed_brand,removed_pack,removed_unit,removed_numbers_and_punctuations,removed_stopwords,cleaned_text


In [9]:
df_train = df_train[~(df_train["cleaned_text"]=="")]
df_test = df_test[~(df_test["cleaned_text"]=="")]

## Create Label

In [10]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()

all_classes = pd.concat([df_train['class'], df_test['class']]).to_frame()

encoder.fit(all_classes)

df_train['label'] = encoder.transform(df_train[['class']])
df_test['label'] = encoder.transform(df_test[['class']])

In [11]:
df_train[df_train["class"] == "laundry detergents"]

,Item_Name,class,Brand,Weight,Number of units,Size of units,Price,T.Price,Pack,Unit,removed_punctuations,removed_numbers,removed_brand,removed_pack,removed_unit,removed_numbers_and_punctuations,removed_stopwords,cleaned_text,label
475,داونى 1 لتر نسيم الوادى زجاجه,laundry detergents,داوني,1لتر,1,NaN,NaN,NaN,زجاجه,لتر,داونى 1 لتر نسيم الوادى زجاجه,داونى لتر نسيم الوادى زجاجه,داونى 1 لتر نسيم الوادى زجاجه,داونى 1 لتر نسيم الوادى,داونى 1 نسيم الوادى زجاجه,داونى لتر نسيم الوادى زجاجه,داونى 1 لتر نسيم الوادى زجاجه,داونى لتر نسيم الوادى زجاجه,23.0
27666,كلوركس ابيض زجاجة,laundry detergents,كلوركس,NaN,1,NaN,NaN,NaN,زجاجة,NaN,كلوركس ابيض زجاجة,كلوركس ابيض زجاجة,ابيض زجاجة,كلوركس ابيض,كلوركس ابيض زجاجة,كلوركس ابيض زجاجة,كلوركس ابيض زجاجة,ابيض زجاجة,23.0


In [12]:
df_train["ara_class"] = df_train["class"].apply(lambda x: CLASSES_TRANSLATION[x])
df_test["ara_class"] = df_test["class"].apply(lambda x: CLASSES_TRANSLATION[x])
df_train["class"] = df_train.apply(lambda x: x["class"] + " " + x["ara_class"], axis=1)
# df_test["class"] = df_test.apply(lambda x: x["class"] + " " + x["ara_class"], axis=1)
# df_train

# Evaluate Model

In [13]:
df_test.dropna(subset=["class", "cleaned_text"], inplace=True)
df_train.dropna(subset=["class", "cleaned_text"], inplace=True)

In [14]:
n_sample = None

In [15]:

multi_column_embedding_model_evaluation(
    df_test,
    ".." / QWEN3_EMBEDDING_CONFIG_PATH,
    [
        # "Item_Name",
        "removed_numbers",
        # "removed_stopwords",
        "cleaned_text",
    ],
    n_sample
)

You are evaluating: hyunseop/qwen3-emb-8b-bnb-int8


  2%|▏         | 100/4772 [02:24<1:57:52,  1.51s/it]

Step 100: Intermediate score = 0.4932


  4%|▍         | 200/4772 [04:46<1:55:29,  1.52s/it]

Step 200: Intermediate score = 0.4567


...

Step 400: Intermediate score = 0.4899


 10%|█         | 500/4772 [11:53<1:47:14,  1.51s/it]

Step 500: Intermediate score = 0.4932


 12%|█▏        | 565/4772 [13:26<1:40:01,  1.43s/it]


KeyboardInterrupt: 

# Save Cleaned Data

In [49]:
df_train.head()

,Item_Name,class,Brand,Weight,Number of units,Size of units,Price,T.Price,Pack,Unit,cleaned_text,label,ara_class
0,مونتانا ذره 400 ج,vegetables fruits الخضروات والفواكه,مونتانا,400جم,1,NaN,NaN,NaN,كيس,جم,ذره,40.0,الخضروات والفواكه
1,ahmad tea fruit and herb selection herbal teab...,tea coffee hot drinks الشاي والقهوة والمشروبات...,ahmad tea,NaN,20,NaN,NaN,NaN,علبة,NaN,fruit herb selection herbal teabags pieces,38.0,الشاي والقهوة والمشروبات الساخنة
2,lulu brown samoon 1pkt,bakery المخبوزات,lulu,NaN,1,NaN,NaN,NaN,عبوة,NaN,brown samoon pkt,1.0,المخبوزات
3,فلفل رومى بلدى حشو وزن,vegetables fruits الخضروات والفواكه,NaN,NaN,1,NaN,NaN,NaN,كيس,NaN,فلفل رومى بلدى حشو وزن,40.0,الخضروات والفواكه
4,كانز,soft drinks juices المشروبات الغازية والعصائر,NaN,NaN,1,NaN,NaN,NaN,كانز,NaN,كانز,33.0,المشروبات الغازية والعصائر


In [50]:
df_train.drop_duplicates(subset=["Item_Name"], inplace=True)
df_train.drop(columns=["Item_Name", "Brand", "Weight", "Number of units", "Size of units", "Price", "T.Price", "Pack", "Unit"], inplace=True)
df_train.dropna(subset=["cleaned_text"])

df_test.drop_duplicates(subset=["Item_Name"], inplace=True)
df_test.drop(columns=["Item_Name", "Brand", "Weight", "Number of units", "Size of units", "Price", "T.Price", "Pack", "Unit"], inplace=True)
df_test.dropna(subset=["cleaned_text"])

df_train

,class,cleaned_text,label,ara_class
0,vegetables fruits الخضروات والفواكه,ذره,40.0,الخضروات والفواكه
1,tea coffee hot drinks الشاي والقهوة والمشروبات...,fruit herb selection herbal teabags pieces,38.0,الشاي والقهوة والمشروبات الساخنة
2,bakery المخبوزات,brown samoon pkt,1.0,المخبوزات
3,vegetables fruits الخضروات والفواكه,فلفل رومى بلدى حشو وزن,40.0,الخضروات والفواكه
4,soft drinks juices المشروبات الغازية والعصائر,كانز,33.0,المشروبات الغازية والعصائر
...,...,...,...,...
42939,tins jars packets المعلبات والعبوات,tomato paste g,39.0,المعلبات والعبوات
42941,tins jars packets المعلبات والعبوات,فول مدمس معلب مقشور بالخلطة المصرية جم,39.0,المعلبات والعبوات
42943,chips crackers رقائق البطاطس والمقرمشات,كباب عائلى,7.0,رقائق البطاطس والمقرمشات
42945,beef processed meat لحم البقر واللحوم المصنعة,لحم وجه فخذ بقر محلي,3.0,لحم البقر واللحوم المصنعة


In [51]:
df_train = df_train[df_train["cleaned_text"]!="nan"]
df_test = df_test[df_test["cleaned_text"]!="nan"]

In [52]:
df_train.to_csv(".." / CLEANED_TRAIN_DATA_PATH, index=False, encoding="utf-8-sig")
df_test.to_csv(".." / CLEANED_TEST_DATA_PATH, index=False, encoding="utf-8-sig")